In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jax.random import PRNGKey
from hyperiax.execution import UnorderedExecutor
from hyperiax.tree.tree import HypTree
from hyperiax.models import UpdateLambda
import jax
from hyperiax.tree.topology import symmetric_topology
from jax import numpy as jnp


In [3]:
key = PRNGKey(1337)
depth = 14
print("Number of nodes:", (2<<depth)-1)

Number of nodes: 32767


In [4]:
topology = symmetric_topology(depth, 2)
tree = HypTree(topology, precompute_child_gathers=True)
#tree = FastBiTree(depth)
tree.add_property('edge_length', (1,))
tree.add_property('value', (1,))
tree.add_property('test_value', (1,))

In [5]:
tree.data['edge_length'] = jnp.ones_like(tree.data['edge_length'])
tree.data['value'] = jnp.arange(len(tree))[:, jnp.newaxis]

In [6]:
def update(child_value, parent_value, value, leaf_mask, **kwargs):
    child_sums = child_value.sum(axis=1)
    child_sums = jnp.where(leaf_mask[:, jnp.newaxis], jnp.zeros_like(child_sums), child_sums)

    return {'test_value': (child_sums+parent_value+value)}

model = UpdateLambda(update)

In [7]:
exe = UnorderedExecutor(model)

In [8]:
exe.update(tree)